# wetterdienst - A basic notebook example

pip install wetterdienst

## Import modules necessary for general functioning

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from wetterdienst.provider.dwd.observation import DwdObservationRequest, \
    DwdObservationPeriod, DwdObservationResolution, DwdObservationParameter, DwdObservationDataset

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

Which parameters are available?

In [ ]:
# all
print("All available parameters")
print(
    DwdObservationRequest.discover()
)
# selection
print("Selection of daily data")
print(
    DwdObservationRequest.discover(
        resolution=DwdObservationResolution.DAILY
    )
)

## 1. First check the metadata to inform yourself of available stations
(here we pick historical daily precipitation - hdp)

In [ ]:
request = DwdObservationRequest(
    parameter=DwdObservationDataset.PRECIPITATION_MORE,
    resolution=DwdObservationResolution.DAILY,
    period=DwdObservationPeriod.HISTORICAL
)
print("Number of stations with available data: ", request.all().df.sum())
print("Some of the stations:")
request.all().df.head()

The metadata includes an id, the range of the measurements, the position
(including height) as well as place and state of it and if it has a file. With the
following plot we want to show a map of those stations:

In [ ]:
cmap = cm.get_cmap('viridis')
bounds = request.all().df.height.quantile([0, 0.25, 0.5, 0.75, 1]).values
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

fig, ax = plt.subplots(figsize=(10, 10))
plot = request.all().df.plot.scatter(
    x="longitude", y="latitude", c="height", cmap=cmap, norm=norm, ax=ax)
plot.set_title("Map of daily precipitation stations in Germany\n"
               "Color refers to height of station")
plt.show()

## 2. The usual way of retrieving data

Let's get some data for the above request

In [ ]:
print("Receiving historical daily climate data for Dresden-Klotzsche (1048)")
station_data = request.filter_by_station_id(station_id=[1048]).values.all().df

station_data.dropna(axis=0).head()

See that DATE is already parsed, so we can easily get some nice graphs with matplotlib,
which we will do in the next part. We can also request direct parameters from the given
parameter sets. We know that CLIMATE_SUMMARY contains TMK, TNK, TXK and RSK, so let's
request those. This option will automatically tidy the data.

In [ ]:
print("Receiving historical daily temperature and precipitation for Dresden-Klotzsche "
      "(1048).")

request = DwdObservationRequest(
    parameter=[
        DwdObservationParameter.DAILY.TEMPERATURE_AIR_200,
        DwdObservationParameter.DAILY.TEMPERATURE_AIR_MAX_200,
        DwdObservationParameter.DAILY.TEMPERATURE_AIR_MIN_200,
        DwdObservationParameter.DAILY.PRECIPITATION_HEIGHT
    ],
    resolution=DwdObservationResolution.DAILY,
    period=DwdObservationPeriod.HISTORICAL
).filter_by_station_id(station_id=(1048, ))

station_data = request.values.all().df

station_data.dropna(axis=0).head()

## 3. Let's create some plots

Now that we have data, let's create some plots! We can create a time series/histogram of
the temperatures and precipitation

In [ ]:
cmap = plt.get_cmap('viridis', 4)
colors = cmap.colors

PARAMETERS = ["tnk", "tmk", "txk", "rsk"]

station_data_grouped = station_data.groupby(station_data["parameter"], observed=True)

fig, axes = plt.subplots(nrows=len(PARAMETERS), tight_layout=True, figsize=(10, 40))

for (parameter, group), ax, color in zip(station_data_grouped, axes, colors):
    group.plot(x="date", y="value", label=parameter, alpha=.75, ax=ax, c=color)

plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.suptitle("Temperature and precipitation time series of Dresden, Germany")

plt.show()

## 4. Create yearly values

In [ ]:
import numpy as np
import pandas as pd
station_data_yearly = []

for (year, parameter), group in station_data.groupby(
        [station_data["date"].dt.year, "parameter"], as_index=False, observed=True):
    if parameter == "rsk":
        station_data_yearly.append(group.dropna().agg({"value": np.sum}))
    else:
        station_data_yearly.append(group.dropna().agg({"value": np.mean}))

station_data_yearly = pd.concat(station_data_yearly)

station_data_yearly

## 5. Find a station

To find a station near to a certain area, call ``filter_by_rank()``.

In [ ]:
DwdObservationRequest(
    parameter=DwdObservationDataset.CLIMATE_SUMMARY,
    resolution=DwdObservationResolution.DAILY,
    period=DwdObservationPeriod.HISTORICAL,
    start_date="2000-01-01",
    end_date="2010-01-01"
).filter_by_rank(
    51.05089,
    13.73832,
    5
).df